# MIDI-Only Piano Performance Evaluation - PercePiano Training

**Goal**: Train MIDI-only model on PercePiano expert labels to validate symbolic-only analysis.

**Target**: R^2 >= 0.185 (match PercePiano Bi-LSTM baseline)

**Stretch Goal**: R^2 >= 0.30 (near best published result of 0.397)

## What You Need on Google Drive

Upload this folder to your Google Drive root:
- `gdrive:percepiano_data/` containing:
  - `percepiano_train.json`
  - `percepiano_val.json`
  - `percepiano_test.json`
  - `PercePiano/` (the cloned repository with MIDI files)

## Step 1: Setup Environment

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install uv and rclone
!curl -LsSf https://astral.sh/uv/install.sh | sh
!curl -fsSL https://rclone.org/install.sh | sudo bash 2>&1 | grep -E "(successfully|already)" || echo "rclone installed"

import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"

# Clone repository
if not os.path.exists('/tmp/crescendai'):
    !git clone https://github.com/Jai-Dhiman/crescendai.git /tmp/crescendai

%cd /tmp/crescendai/model
!git pull
!git log -1 --oneline

# Install dependencies
!uv pip install --system -e .
!pip install tensorboard rich

import torch
import pytorch_lightning as pl
print(f"\nPyTorch: {torch.__version__}")
print(f"Lightning: {pl.__version__}")

## Step 2: Configure rclone (Run Once)

Run this in a terminal:
```bash
rclone config
```

Follow prompts to set up `gdrive` remote for Google Drive.

In [ ]:
import os
from pathlib import Path
import subprocess

# Paths
CHECKPOINT_ROOT = '/tmp/checkpoints/midi_only_percepiano'
GDRIVE_CHECKPOINT_PATH = 'gdrive:crescendai_checkpoints/midi_only_percepiano'
GDRIVE_DATA_PATH = 'gdrive:percepiano_data'
DATA_ROOT = Path('/tmp/percepiano_data')

print("="*70)
print("SETUP: CHECKPOINTS AND DATA")
print("="*70)

# Create directories
os.makedirs(CHECKPOINT_ROOT, exist_ok=True)
DATA_ROOT.mkdir(parents=True, exist_ok=True)

# Check rclone
print("\nChecking rclone configuration...")
result = subprocess.run(['rclone', 'listremotes'], capture_output=True, text=True)

if 'gdrive:' in result.stdout:
    print("  rclone 'gdrive' remote: CONFIGURED")
    RCLONE_AVAILABLE = True
    
    # Restore existing checkpoints
    print("\nRestoring checkpoints from Google Drive (if any)...")
    subprocess.run(
        ['rclone', 'copy', GDRIVE_CHECKPOINT_PATH, CHECKPOINT_ROOT, '--progress'],
        capture_output=False
    )
else:
    print("  rclone 'gdrive' remote: NOT CONFIGURED")
    print("  Run 'rclone config' in terminal to set up Google Drive")
    RCLONE_AVAILABLE = False

print(f"\nCheckpoint directory: {CHECKPOINT_ROOT}")
print(f"rclone available: {RCLONE_AVAILABLE}")

## Step 3: Download PercePiano Data

In [ ]:
from pathlib import Path
import subprocess
import json

DATA_ROOT = Path('/tmp/percepiano_data')
DATA_ROOT.mkdir(parents=True, exist_ok=True)

# Check if data already exists
train_file = DATA_ROOT / 'percepiano_train.json'
if train_file.exists():
    print(f"Data already exists at {DATA_ROOT}")
else:
    print("Downloading PercePiano data from Google Drive...")
    result = subprocess.run(
        ['rclone', 'copy', GDRIVE_DATA_PATH, str(DATA_ROOT), '--progress'],
        capture_output=False
    )

# Verify data
for split in ['train', 'val', 'test']:
    path = DATA_ROOT / f'percepiano_{split}.json'
    if path.exists():
        with open(path) as f:
            data = json.load(f)
        print(f"{split}: {len(data)} samples")
    else:
        print(f"ERROR: {path} not found!")

# Check MIDI files
midi_dir = DATA_ROOT / 'PercePiano' / 'virtuoso' / 'data' / 'all_2rounds'
if midi_dir.exists():
    midi_files = list(midi_dir.glob('*.mid'))
    print(f"\nMIDI files: {len(midi_files)}")
else:
    print(f"\nERROR: MIDI directory not found at {midi_dir}")
    print("Make sure to upload the full PercePiano repository")

## Step 4: Regenerate JSON Files with 19 Dimensions

The dataset needs to be regenerated to use all 19 PercePiano dimensions (matching the reference implementation).

In [ ]:
import json
from pathlib import Path

DATA_ROOT = Path('/tmp/percepiano_data')

# All 19 PercePiano dimensions (matching reference implementation)
PERCEPIANO_DIMENSIONS = [
    "timing",              # 0: Stable <-> Unstable
    "articulation_length", # 1: Short <-> Long
    "articulation_touch",  # 2: Soft/Cushioned <-> Hard/Solid
    "pedal_amount",        # 3: Sparse/Dry <-> Saturated/Wet
    "pedal_clarity",       # 4: Clean <-> Blurred
    "timbre_variety",      # 5: Even <-> Colorful
    "timbre_depth",        # 6: Shallow <-> Rich
    "timbre_brightness",   # 7: Bright <-> Dark
    "timbre_loudness",     # 8: Soft <-> Loud
    "dynamic_range",       # 9: Little Range <-> Large Range
    "tempo",               # 10: Fast-paced <-> Slow-paced
    "space",               # 11: Flat <-> Spacious
    "balance",             # 12: Disproportioned <-> Balanced
    "drama",               # 13: Pure <-> Dramatic
    "mood_valence",        # 14: Optimistic <-> Dark
    "mood_energy",         # 15: Low Energy <-> High Energy
    "mood_imagination",    # 16: Honest <-> Imaginative
    "sophistication",      # 17: Sophisticated/Mellow <-> Raw/Crude
    "interpretation",      # 18: Unsatisfactory <-> Convincing
]

def regenerate_with_19_dims(data_root: Path):
    """Regenerate JSON files with all 19 dimensions in 0-1 scale."""
    
    for split in ['train', 'val', 'test']:
        path = data_root / f'percepiano_{split}.json'
        
        with open(path) as f:
            data = json.load(f)
        
        for sample in data:
            # Get original PercePiano scores (first 19 values, 20th is performer ID)
            pp_scores = sample['percepiano_scores'][:19]
            
            # Map to all 19 dimensions with original 0-1 scale
            sample['scores'] = {
                dim: pp_scores[i]
                for i, dim in enumerate(PERCEPIANO_DIMENSIONS)
            }
            
            # Update MIDI path to Thunder Compute location
            filename = Path(sample['midi_path']).name
            sample['midi_path'] = str(data_root / 'PercePiano' / 'virtuoso' / 'data' / 'all_2rounds' / filename)
        
        # Save updated file
        with open(path, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"Regenerated {split}: {len(data)} samples with 19 dimensions")

# Regenerate the JSON files
regenerate_with_19_dims(DATA_ROOT)

# Verify the new format
with open(DATA_ROOT / 'percepiano_train.json') as f:
    data = json.load(f)

sample = data[0]
print(f"\nSample dimensions: {len(sample['scores'])}")
print(f"Dimension names: {list(sample['scores'].keys())}")
print(f"Sample scores (first 5):")
for i, (dim, score) in enumerate(sample['scores'].items()):
    if i >= 5:
        break
    print(f"  {dim}: {score:.4f}")

# Verify MIDI path
sample_path = Path(sample['midi_path'])
print(f"\nMIDI path: {sample_path}")
print(f"Exists: {sample_path.exists()}")

## Step 5: Training Configuration

In [ ]:
import torch
torch.set_float32_matmul_precision('medium')  # Tensor Core optimization

CONFIG = {
    # Data
    'data_dir': '/tmp/percepiano_data',
    
    # Model Architecture - UPDATED TO MATCH MidiBERT (768/12/12)
    'midi_hidden_dim': 768,    # Increased from 256 to match MidiBERT
    'midi_num_layers': 12,     # Increased from 6 to match MidiBERT
    'midi_num_heads': 12,      # Increased from 8 to match MidiBERT
    'max_seq_length': 512,     # Match PercePiano reference
    'attention_da': 128,       # PercePiano self-attention hidden dim
    'attention_r': 4,          # PercePiano self-attention heads
    'head_hidden_dim': 256,    # PercePiano classifier hidden dim
    'dropout': 0.1,
    'use_percepiano_architecture': True,  # Use new PercePiano-style components
    
    # Legacy params (only used if use_percepiano_architecture=False)
    'lstm_hidden': 256,
    'lstm_layers': 2,
    'attention_heads': 4,
    'shared_hidden': 256,
    'task_hidden': 128,
    
    # Training - MATCHED TO PERCEPIANO REFERENCE
    'batch_size': 8,           # Reduced for larger model (was 12)
    'learning_rate': 1e-5,     # PercePiano uses 1e-5
    'weight_decay': 0.01,
    'max_epochs': 100,         # PercePiano uses 100
    'early_stopping_patience': 20,  # PercePiano uses 20
    'gradient_clip_val': 1.0,
    'precision': '16-mixed',
    
    # Checkpoints
    'checkpoint_dir': '/tmp/checkpoints/midi_only_percepiano',
    'gdrive_checkpoint': 'gdrive:crescendai_checkpoints/midi_only_percepiano',
}

# Print config
print("Training Configuration (PercePiano Architecture Overhaul):")
print("="*60)
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

## Step 6: Create DataLoaders

In [ ]:
from pathlib import Path
from src.data.percepiano_dataset import create_dataloaders

train_loader, val_loader, test_loader = create_dataloaders(
    data_dir=Path(CONFIG['data_dir']),
    batch_size=CONFIG['batch_size'],
    max_seq_length=CONFIG['max_seq_length'],
    num_workers=4,
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

# Test a batch
batch = next(iter(train_loader))
print(f"\nBatch shapes:")
print(f"  midi_tokens: {batch['midi_tokens'].shape}")
print(f"  attention_mask: {batch['attention_mask'].shape}")
print(f"  scores: {batch['scores'].shape}")

## Step 6b: (Optional) MAESTRO Pretraining

**Skip this section if you want to train from scratch** - just run the cells below.

To pretrain the encoder on MAESTRO for better performance:
1. Download MAESTRO v3.0 MIDI files from https://magenta.tensorflow.org/datasets/maestro
2. Upload to `gdrive:maestro_midi/` (just the MIDI folder, ~85MB)
3. Run the cells below

Pretraining takes ~8-12 hours on A100 but can significantly improve results.

In [ ]:
# Download pretrained encoder from Google Drive
import subprocess
from pathlib import Path
import os

PRETRAIN_CHECKPOINT_DIR = Path('/tmp/checkpoints/midi_pretrain')
PRETRAIN_GDRIVE = 'gdrive:crescendai_checkpoints/midi_pretrain'

PRETRAIN_CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("PRETRAINED ENCODER SETUP")
print("=" * 70)

# Download pretrained encoder (this is the key file - 344MB)
print("\nDownloading pretrained encoder from Google Drive...")
result = subprocess.run(
    ['rclone', 'copy', f'{PRETRAIN_GDRIVE}/encoder_pretrained.pt', str(PRETRAIN_CHECKPOINT_DIR), '--progress'],
    capture_output=False
)

# Verify download
pretrained_path = PRETRAIN_CHECKPOINT_DIR / 'encoder_pretrained.pt'
if pretrained_path.exists():
    size_mb = pretrained_path.stat().st_size / (1024 * 1024)
    print(f"\nPretrained encoder downloaded successfully!")
    print(f"  Path: {pretrained_path}")
    print(f"  Size: {size_mb:.1f} MB")
    
    # Verify it's the right file (should be ~330MB for 768/12/12 encoder)
    if size_mb < 100:
        print(f"\n  WARNING: File seems too small ({size_mb:.1f} MB). Expected ~330 MB.")
        print("  The encoder may not have downloaded correctly.")
    else:
        print(f"  Status: OK (size looks correct for 768/12/12 encoder)")
    
    PRETRAINED_EXISTS = True
else:
    print("\nERROR: Pretrained encoder not found after download attempt!")
    print("Check that gdrive:crescendai_checkpoints/midi_pretrain/encoder_pretrained.pt exists")
    PRETRAINED_EXISTS = False

print("=" * 70)

In [ ]:
# This cell is now optional - pretraining has already been completed
# The pretrained encoder (encoder_pretrained.pt) was downloaded in the previous cell

# Only run this if you need to RE-pretrain from scratch (takes 8-12 hours)
RUN_PRETRAINING = False  # Set to True only if you want to redo pretraining

if RUN_PRETRAINING:
    print("Pretraining is disabled. Set RUN_PRETRAINING=True to re-pretrain.")
    print("Note: Pretraining takes 8-12 hours on A100.")
else:
    if PRETRAINED_EXISTS:
        print("Using existing pretrained encoder from GDrive (recommended)")
        print(f"  File: {pretrained_path}")
    else:
        print("WARNING: No pretrained encoder available.")
        print("The model will train from random initialization (lower performance expected).")

## Step 7: Create Model

In [ ]:
from src.models.midi_only_module import MIDIOnlyModule

# Create model with explicit architecture matching pretrained encoder (768/12/12)
model = MIDIOnlyModule(
    # Encoder params - MUST match pretrained encoder (768/12/12)
    midi_hidden_dim=CONFIG['midi_hidden_dim'],
    midi_num_layers=CONFIG['midi_num_layers'],
    midi_num_heads=CONFIG['midi_num_heads'],
    max_seq_length=CONFIG['max_seq_length'],
    # PercePiano self-attention aggregation
    attention_da=CONFIG['attention_da'],
    attention_r=CONFIG['attention_r'],
    # PercePiano classifier head
    head_hidden_dim=CONFIG['head_hidden_dim'],
    # Training params
    learning_rate=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay'],
    dropout=CONFIG['dropout'],
    # Architecture selection
    use_percepiano_architecture=CONFIG['use_percepiano_architecture'],
    # Legacy params (only used if use_percepiano_architecture=False)
    lstm_hidden=CONFIG['lstm_hidden'],
    lstm_layers=CONFIG['lstm_layers'],
    attention_heads=CONFIG['attention_heads'],
    shared_hidden=CONFIG['shared_hidden'],
    task_hidden=CONFIG['task_hidden'],
)

# Verify architecture matches config
print("=" * 70)
print("MODEL ARCHITECTURE VERIFICATION")
print("=" * 70)
print(f"Expected: {CONFIG['midi_hidden_dim']}d / {CONFIG['midi_num_layers']}L / {CONFIG['midi_num_heads']}H")
print(f"Actual:   {model.midi_encoder.hidden_size}d / {model.midi_encoder.num_layers}L / {model.midi_encoder.transformer.layers[0].self_attn.num_heads}H")
print(f"PercePiano architecture: {CONFIG['use_percepiano_architecture']}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print("=" * 70)

# Verify config values are correct (not defaults)
assert CONFIG['midi_hidden_dim'] == 768, f"ERROR: midi_hidden_dim should be 768, got {CONFIG['midi_hidden_dim']}"
assert CONFIG['midi_num_layers'] == 12, f"ERROR: midi_num_layers should be 12, got {CONFIG['midi_num_layers']}"
assert CONFIG['use_percepiano_architecture'] == True, "ERROR: use_percepiano_architecture should be True"

# Load pretrained encoder weights if available
pretrained_path = PRETRAIN_CHECKPOINT_DIR / 'encoder_pretrained.pt'
if pretrained_path.exists():
    print(f"\nLoading pretrained encoder from {pretrained_path}...")
    model.midi_encoder.load_pretrained(pretrained_path)
    print("Pretrained weights loaded successfully!")
else:
    print("\n" + "!" * 70)
    print("WARNING: No pretrained encoder found - training from scratch!")
    print("This will likely result in lower performance.")
    print("!" * 70)

print(f"\nDimensions: {model.dimensions}")

## Step 8: Setup Training

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import subprocess
from pathlib import Path
import os

# Checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath=CONFIG['checkpoint_dir'],
    filename='midi_only-{epoch:02d}-{val_mean_r:.3f}',
    monitor='val/mean_r',
    mode='max',
    save_top_k=3,
    save_last=True,
)

# Early stopping
early_stopping = EarlyStopping(
    monitor='val/mean_r',
    patience=CONFIG['early_stopping_patience'],
    mode='max',
)

# LR monitor
lr_monitor = LearningRateMonitor(logging_interval='step')

# Logger
logger = TensorBoardLogger(
    save_dir='/tmp/logs',
    name='midi_only_percepiano',
)

# Trainer
trainer = pl.Trainer(
    max_epochs=CONFIG['max_epochs'],
    accelerator='gpu',
    devices=1,
    precision=CONFIG['precision'],
    gradient_clip_val=CONFIG['gradient_clip_val'],
    callbacks=[checkpoint_callback, early_stopping, lr_monitor],
    logger=logger,
    log_every_n_steps=10,
    deterministic=True,
)

print("Trainer configured!")
print(f"  Precision: {CONFIG['precision']}")
print(f"  Max epochs: {CONFIG['max_epochs']}")
print(f"  Early stopping patience: {CONFIG['early_stopping_patience']}")

## Step 9: Train!

In [ ]:
# Set seed for reproducibility
pl.seed_everything(42, workers=True)

# Train
print("Starting training...")
trainer.fit(model, train_loader, val_loader)

In [ ]:
# Sync checkpoints to Google Drive
if RCLONE_AVAILABLE:
    print("Syncing checkpoints to Google Drive...")
    subprocess.run(
        ['rclone', 'copy', CONFIG['checkpoint_dir'], CONFIG['gdrive_checkpoint'], '--progress'],
        capture_output=False
    )
    print("Sync complete!")

## Step 10: Apply Post-Hoc Calibration

If Pearson r is positive but R^2 is negative, the model has learned relative ranking but predictions are systematically biased. Calibration fixes this.

In [ ]:
# Apply calibration to fix systematic bias
print("Applying post-hoc calibration...")

# Load best model
from src.models.midi_only_module import MIDIOnlyModule
best_model = MIDIOnlyModule.load_from_checkpoint(checkpoint_callback.best_model_path)
best_model.eval()
best_model.cuda()

# Calibrate on validation set
calibration_results = best_model.calibrate(val_loader, method='both')

print("\nCalibration Results:")
print("="*60)
print(f"Baseline R^2:     {calibration_results['baseline_r2']:.4f}")
print(f"Baseline MSE:     {calibration_results['baseline_mse']:.6f}")
print(f"---")
print(f"Isotonic R^2:     {calibration_results['isotonic_r2']:.4f}")
print(f"Isotonic MSE:     {calibration_results['isotonic_mse']:.6f}")
print(f"---")
print(f"Temperature R^2:  {calibration_results['temperature_r2']:.4f}")
print(f"Temperature MSE:  {calibration_results['temperature_mse']:.6f}")
print(f"Temperature value: {calibration_results['temperature_value']:.4f}")
print(f"---")
print(f"Selected method:  {calibration_results['selected_method']}")
print("="*60)

# Check improvement
r2_before = calibration_results['baseline_r2']
r2_after = calibration_results.get(f"{calibration_results['selected_method']}_r2", r2_before)
if r2_after > r2_before:
    print(f"\nCalibration improved R^2 by {r2_after - r2_before:.4f}")
else:
    print(f"\nCalibration did not improve R^2 (model may already be well-calibrated)")

In [ ]:
# Test with best checkpoint
print("\nRunning test with best checkpoint...")
best_path = checkpoint_callback.best_model_path
print(f"Best checkpoint: {best_path}")

if best_path:
    test_results = trainer.test(model, test_loader, ckpt_path=best_path)
    print("\nTest Results:")
    for k, v in test_results[0].items():
        print(f"  {k}: {v:.4f}")

## Step 11: Analyze Results

In [ ]:
import torch
import numpy as np
from scipy import stats

# Load best model
best_model = MIDIOnlyModule.load_from_checkpoint(checkpoint_callback.best_model_path)
best_model.eval()
best_model.cuda()

# Collect predictions
all_preds = []
all_targets = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        outputs = best_model(batch['midi_tokens'], batch['attention_mask'])
        all_preds.append(outputs['predictions'].cpu())
        all_targets.append(batch['scores'].cpu())

all_preds = torch.cat(all_preds).numpy()
all_targets = torch.cat(all_targets).numpy()

print("Per-Dimension Results:")
print("="*60)
print(f"{'Dimension':<20} {'Pearson r':<12} {'R^2':<12} {'MAE':<12}")
print("-"*60)

dimensions = best_model.dimensions
results = {}

for i, dim in enumerate(dimensions):
    preds = all_preds[:, i]
    targets = all_targets[:, i]
    
    # Pearson correlation
    r, p = stats.pearsonr(preds, targets)
    
    # R-squared
    ss_res = np.sum((targets - preds) ** 2)
    ss_tot = np.sum((targets - np.mean(targets)) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0
    
    # MAE
    mae = np.mean(np.abs(preds - targets))
    
    results[dim] = {'r': r, 'r2': r2, 'mae': mae}
    print(f"{dim:<20} {r:<12.4f} {r2:<12.4f} {mae:<12.4f}")

# Overall metrics
mean_r = np.mean([v['r'] for v in results.values()])
mean_r2 = np.mean([v['r2'] for v in results.values()])
mean_mae = np.mean([v['mae'] for v in results.values()])

print("-"*60)
print(f"{'MEAN':<20} {mean_r:<12.4f} {mean_r2:<12.4f} {mean_mae:<12.4f}")
print("="*60)

# Compare to baselines
print("\nComparison to PercePiano Baselines:")
print(f"  Bi-LSTM baseline: R^2 = 0.185")
print(f"  MidiBERT:         R^2 = 0.313")
print(f"  Best (HAN):       R^2 = 0.397")
print(f"  Our model:        R^2 = {mean_r2:.3f}")

if mean_r2 >= 0.185:
    print("\n  SUCCESS: Meets baseline target!")
if mean_r2 >= 0.30:
    print("  SUCCESS: Meets stretch goal!")

## Step 12: Save Final Model and Sync

In [ ]:
import torch
from pathlib import Path

# Save final model for inference
final_path = Path(CONFIG['checkpoint_dir']) / 'midi_scorer_final.pt'
torch.save({
    'state_dict': best_model.state_dict(),
    'hparams': dict(best_model.hparams),
    'dimensions': best_model.dimensions,
    'results': results,
    'mean_r2': mean_r2,
    'mean_r': mean_r,
}, final_path)
print(f"Saved final model to {final_path}")

# Final sync to Google Drive
if RCLONE_AVAILABLE:
    print("\nFinal sync to Google Drive...")
    subprocess.run(
        ['rclone', 'copy', CONFIG['checkpoint_dir'], CONFIG['gdrive_checkpoint'], '--progress'],
        capture_output=False
    )
    print("Sync complete!")
    print(f"Checkpoints available at: {CONFIG['gdrive_checkpoint']}")